<a href="https://colab.research.google.com/github/Parag-dwn/fire_detection/blob/main/fire_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install patool

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 3.4 MB/s eta 0:00:00


In [ ]:
import patoolib

In [ ]:
file=patoolib.extract_archive('/content/Fire detection.v2i.coco.zip')

patool: Extracting /content/Fire detection.v2i.coco.zip ...
patool: running /usr/bin/7z x -o./Unpack_fs83lb3h -- "/content/Fire detection.v2i.coco.zip"
patool: ... /content/Fire detection.v2i.coco.zip extracted to `Fire detection.v2i.coco' (multiple files in root).


In [1]:
# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
import pandas as pd
import tensorflow as tf


#Clone Git Repository
!git clone https://github.com/fizyr/keras-retinanet.git
%cd keras-retinanet/
!python setup.py build_ext --inplace

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet import models

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 6224, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 6224 (delta 6), reused 6 (delta 2), pack-reused 6205
Receiving objects: 100% (6224/6224), 13.48 MiB | 27.34 MiB/s, done.
Resolving deltas: 100% (4206/4206), done.
/content/keras-retinanet
/usr/local/lib/python3.9/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/us

In [2]:
#Downloading data from Roboflow
#UPDATE THIS LINK - get our data from Roboflow
%cd /content
!curl -L "https://app.roboflow.com/ds/UqsEJpchkX?key=GLMTc9PTpR" > roboflow.zip; unzip -qq roboflow.zip; rm roboflow.zip

/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   891  100   891    0     0   2930      0 --:--:-- --:--:-- --:--:--  2930
100 32.7M  100 32.7M    0     0  29.2M      0  0:00:01  0:00:01 --:--:-- 62.1M


In [3]:
#Set up paths 

#Path to where the Images are stored
TRAIN_IMAGE_PATH = "/content/train"
VALID_IMAGE_PATH = "/content/valid"
TEST_IMAGE_PATH  = "/content/test"
#Path to where annotations are stored
TRAIN_ANNOT_PATH = "/content/train"
VALID_ANNOT_PATH = "/content/valid"
TEST_ANNOT_PATH  = "/content/test"

In [5]:
%pwd

'/content'

In [6]:
import pandas as pd
from PIL import Image
import cv2
import numpy as np



pd.set_option("display.max_colwidth", None)
np.random.seed(123)

In [7]:
pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import glob
import logging
import os
import xml.etree.ElementTree as ET
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

def convert_annotations_to_df(annotation_dir, image_dir, image_set="train"):
    xml_list = []
    for xml_file in glob.glob(annotation_dir + "/*.xml"):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall("object"):
            bbx = member.find("bndbox")
            xmin = int(bbx.find("xmin").text)
            ymin = int(bbx.find("ymin").text)
            xmax = int(bbx.find("xmax").text)
            ymax = int(bbx.find("ymax").text)
            label = member.find("name").text

            value = (
                root.find("filename").text,
                int(root.find("size")[0].text),
                int(root.find("size")[1].text),
                label,
                xmin,
                ymin,
                xmax,
                ymax,
            )
            xml_list.append(value)

    column_name = [
        "filename",
        "width",
        "height",
        "class",
        "xmin",
        "ymin",
        "xmax",
        "ymax",
    ]

    xml_df = pd.DataFrame(xml_list, columns=column_name)
    xml_df["filename"] = [
        os.path.join(image_dir, xml_df["filename"][i]) for i in range(len(xml_df))
    ]

    if image_set == "train":
        # label encoder encodes the labels from 0
        # we need to add +1 so that labels are encode from 1 as our
        # model reserves 0 for background class.
        xml_df["labels"] = encoder.fit_transform(xml_df["class"]) + 1
    elif image_set == "val" or image_set == "test":
        xml_df["labels"] = encoder.transform(xml_df["class"]) + 1
    return xml_df


In [9]:
#convert xml files to pandas DataFrames
train_df = convert_annotations_to_df(TRAIN_ANNOT_PATH, TRAIN_IMAGE_PATH, image_set="train")
valid_df = convert_annotations_to_df(VALID_ANNOT_PATH, VALID_IMAGE_PATH, image_set="test")
test_df  = convert_annotations_to_df(TEST_ANNOT_PATH, TEST_IMAGE_PATH, image_set="test")


def remove_invalid_annots(df):
    """
    Removes annotations where xmax, ymax < xmin,ymin
    from the given dataframe
    """
    df = df[df.xmax > df.xmin]
    df = df[df.ymax > df.ymin]
    df.reset_index(inplace=True, drop=True)
    return df

# removing annotations that are not valid annotations
train_df = remove_invalid_annots(train_df)
valid_df = remove_invalid_annots(valid_df)
test_df  = remove_invalid_annots(test_df)

In [10]:
train_df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,labels
0,/content/train/img_442_jpg.rf.88236f424368d5fe216a4400926bdb9f.jpg,640,640,fire,129,1,641,347,1
1,/content/train/telugu-desam-office-fire_0_jpg.rf.105ca1dbbe87b4010d5a4afa12f814da.jpg,640,640,fire,163,108,598,578,1
2,/content/train/telugu-desam-office-fire_0_jpg.rf.105ca1dbbe87b4010d5a4afa12f814da.jpg,640,640,fire,1,141,261,492,1
3,/content/train/canada-wildfire_jpg.rf.f01d3d0284665bfd87df09e697147d07.jpg,640,640,fire,338,331,436,478,1
4,/content/train/img_363_jpg.rf.034057ffd9b939255e6265b406b25f72.jpg,640,640,fire,384,230,571,543,1


In [11]:
!pip install --upgrade git+https://github.com/broadinstitute/keras-resnet
import keras
import keras_resnet
import urllib.request

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/broadinstitute/keras-resnet to /tmp/pip-req-build-g5ztb3g1
  Running command git clone --filter=blob:none --quiet https://github.com/broadinstitute/keras-resnet /tmp/pip-req-build-g5ztb3g1
  Resolved https://github.com/broadinstitute/keras-resnet to commit 898a1ee417e940ff12bf73ad44c2aae88501771f
  Preparing metadata (setup.py) ... done
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=22164 sha256=be089abfdb67b7ece3c733bc2b1354dfc308bc9e03d7f3bac09b066a87e76ef7
  Stored in directory: /tmp/pip-ephem-wheel-cache-hrqka867/wheels/2e/30/08/8d388b3e1bf1eebd821cb082f4c20a20c2f6f01c3e6e0a8e3c
Successfully built keras-resnet


In [12]:
#Paths where to save the generated dataframes
TRAIN_CSV = "/content/train_data.csv"
VALID_CSV = "/content/valid_data.csv"
TEST_CSV  = "/content/test_data.csv"

# #Save the dataframes to memory
train_df.to_csv(TRAIN_CSV, index=False)
valid_df.to_csv(VALID_CSV, index=False)
test_df.to_csv(TEST_CSV, index=False)

In [13]:
df_extrain_main=pd.DataFrame(pd.json_normalize(df_extrain['annotations']), columns=['x', 'y', 'width', 'height']).join(df_extrain)
df_extrain_main['class']='fire'
df_extrain_main=df_extrain_main[['image_path','x','y','width','height','class','video_id','video_frame']]
df_extrain_main.head(10)

NameError: ignored

In [14]:
train_df = pd.read_csv(TRAIN_CSV)
valid_df = pd.read_csv(VALID_CSV)
test_df  = pd.read_csv(TEST_CSV)

In [15]:
train_df

,filename,width,height,class,xmin,ymin,xmax,ymax,labels
0,/content/train/img_442_jpg.rf.88236f424368d5fe216a4400926bdb9f.jpg,640,640,fire,129,1,641,347,1
1,/content/train/telugu-desam-office-fire_0_jpg.rf.105ca1dbbe87b4010d5a4afa12f814da.jpg,640,640,fire,163,108,598,578,1
2,/content/train/telugu-desam-office-fire_0_jpg.rf.105ca1dbbe87b4010d5a4afa12f814da.jpg,640,640,fire,1,141,261,492,1
3,/content/train/canada-wildfire_jpg.rf.f01d3d0284665bfd87df09e697147d07.jpg,640,640,fire,338,331,436,478,1
4,/content/train/img_363_jpg.rf.034057ffd9b939255e6265b406b25f72.jpg,640,640,fire,384,230,571,543,1
...,...,...,...,...,...,...,...,...,...
904,/content/train/img_711_jpg.rf.c07dbdbc4f798f437727efa5fa9e9a4c.jpg,640,640,fire,1,18,130,225,1
905,/content/train/dormfire_1_jpg.rf.0719bda54fc40c35be37e49df4126829.jpg,640,640,fire,16,1,641,534,1
906,/content/train/img_245_jpg.rf.aaa3364732b6e567d07352e8a2e3045a.jpg,640,640,fire,1,124,158,533,1
907,/content/train/Chimney-Fire3_jpg.rf.95c4bec765da910566e5a244de77d7d8.jpg,640,640,fire,198,288,326,451,1


In [17]:
%cd keras-retinanet/
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'
#### OPTION 1: DOWNLOAD INITIAL PRETRAINED MODEL FROM FIZYR ####
URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)
print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

/content/keras-retinanet
Downloaded pretrained model to ./snapshots/_pretrained_model.h5


In [18]:
import tensorflow as tf
import contextlib2
import io
import IPython
import json
import numpy as np
import os
import pathlib
import pandas as pd
import sys
import tensorflow as tf
import time
classes=pd.DataFrame([{'class':'fire','label':0}])
classes.to_csv("classes.csv",index=False,header=False)

In [19]:
train_df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,labels
0,/content/train/img_442_jpg.rf.88236f424368d5fe216a4400926bdb9f.jpg,640,640,fire,129,1,641,347,1
1,/content/train/telugu-desam-office-fire_0_jpg.rf.105ca1dbbe87b4010d5a4afa12f814da.jpg,640,640,fire,163,108,598,578,1
2,/content/train/telugu-desam-office-fire_0_jpg.rf.105ca1dbbe87b4010d5a4afa12f814da.jpg,640,640,fire,1,141,261,492,1
3,/content/train/canada-wildfire_jpg.rf.f01d3d0284665bfd87df09e697147d07.jpg,640,640,fire,338,331,436,478,1
4,/content/train/img_363_jpg.rf.034057ffd9b939255e6265b406b25f72.jpg,640,640,fire,384,230,571,543,1


In [20]:
classes=pd.DataFrame([{'class':'fire','label':0}])
classes.to_csv("classes.csv",index=False,header=False)
train_df['class']='fire'
train_df[['filename','xmin','ymin','width','height','class']].to_csv("annotation.csv",index=False,header=False)

In [21]:
pip install tensorrt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.2/819.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.6/719.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.7 MB/s eta 0:00:00


In [23]:
!keras_retinanet/bin/train.py --freeze-backbone --random-transform --weights {PRETRAINED_MODEL} --batch-size 1 --steps 100 --epochs 10 csv annotation.csv classes.csv

2023-04-25 02:28:40.753759: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Creating model, this may take a second...
/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
Model: "retinanet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                              

In [24]:
model_path = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[0])
#print(model_path)

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')  ## Use backbone as resnet50
model = models.convert_model(model)

# load label to names mapping for visualization purposes
labels_to_names = pd.read_csv('classes.csv',header=None).T.loc[0].to_dict()

/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [28]:
THRES_SCORE = 0.4  # Set Score Threshold Value

def df_plot_orinal(drawOG,img_path,df):
    df=df[df['filename']==img_path]
    for i,r in df.iterrows():
        cv2.rectangle(drawOG, (r['x'], r['y']), (r['width'], r['height']), (255,0,0),2)
    

def img_inference(img_path):
  image = read_image_bgr(img_path)

  # copy to draw on
  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
  drawOG = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  # preprocess image for network
  image = preprocess_image(image)
  image, scale = resize_image(image)

  # process image
  start = time.time()
  boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
  df_plot_orinal(drawOG,img_path,train_df)
  # correct for image scale
  boxes /= scale
  # visualize detections
  for box, score, label in zip(boxes[0], scores[0], labels[0]):
      # scores are sorted so we can break
      #print(score)
      if score < THRES_SCORE:
          continue
      color = label_color(label)
      b = box.astype(int)
      draw_box(draw, b, color=color)
      caption = "{} {:.3f}%".format(labels_to_names[label], score*100)
    
  fig = plt.figure(figsize=(20, 20))
  ax1=fig.add_subplot(1, 2, 1)
  plt.imshow(draw)
  ax2=fig.add_subplot(1, 2, 2)
  plt.imshow(drawOG)

  ax1.title.set_text('Predicted')
  ax2.title.set_text('Actual')
  plt.show()

In [29]:
data=train_df.sample(n=5)  #Predict on Random 5 Image
for i,r in data.iterrows():
    img_inference(r['filename'])

KeyError: ignored

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="8vnBWj0WovA1omeD2rY8")
project = rf.workspace("parag-dewangan-4ba9o").project("fire-detection-35gdj")
dataset = project.version(2).download("retinanet")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.6 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=56e98d795fcbd0a61fe8f4540bc73c2c7a1006e89c07157390b0f353554efc80
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
    

loading Roboflow workspace...
loading Roboflow project...

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="API_KEY")
project = rf.workspace().project("MODEL_ENDPOINT")
model = project.version(VERSION).model

# infer on a local image
print(model.predict("your_image.jpg", confidence=40, overlap=30).json())

# visualize your prediction
# model.predict("your_image.jpg", confidence=40, overlap=30).save("prediction.jpg")

# infer on an image hosted elsewhere
# print(model.predict("URL_OF_YOUR_IMAGE", hosted=True, confidence=40, overlap=30).json())  